In [1]:
import pandas as pd
import pintpandas

DISTRICT_HEATING_OPERATOR = '005'  # Helen Oy
INPUT_DATASETS = [
    'jyrjola/energiateollisuus/district_heating_fuel',
    'jyrjola/energiateollisuus/district_heating_production',
    'jyrjola/statfi/fuel_classification'
]
from utils.quilt import load_datasets

dh_fuel_df, dh_production_df, fuel_classification = load_datasets(INPUT_DATASETS)

In [2]:
fuel_co2 = fuel_classification[['code', 'co2e_emission_factor', 'is_bio']].set_index('code')
df = dh_fuel_df[dh_fuel_df.Operator == DISTRICT_HEATING_OPERATOR]
df = df.merge(fuel_co2, how='left', left_on='StatfiFuelCode', right_index=True)
df.co2e_emission_factor = df.co2e_emission_factor.astype('pint[t/TJ]')
df.Value = df.Value.astype('pint[GWh]')
operator_fuel_with_co2 = df.copy()
df['Emissions'] = (df.Value * df.co2e_emission_factor).pint.to('tonne').pint.m

df.loc[df.is_bio == True, 'Emissions'] = 0
emissions = df[df.Operator == DISTRICT_HEATING_OPERATOR].groupby('Year').Emissions.sum()
emissions.name = 'Emissions'

In [11]:
df = dh_production_df
df = df[df.Operator == DISTRICT_HEATING_OPERATOR]
#display(df)
heat_production = df[df.Quantity == 'Käyttö'].set_index('Year').Value
heat_production.name = 'Heat production'
electricity_production = df[df.Quantity == 'Kaukolämmön tuotantoon liittyvä sähkön nettotuotanto'].set_index('Year').Value
electricity_production.name = 'Electricity production'

# Determine the CHP alternate production energy consumptions according to the efficiency method
electricity_production_alternate = electricity_production / 0.39
heat_production_alternate = heat_production / 0.90
total = electricity_production_alternate + heat_production_alternate
heat_share = heat_production_alternate / total
heat_share.name = 'Share of heat production'


Year
2005    20760.846154
2010    22525.213675
2014    19464.923077
2015    18607.247863
2016    17566.957265
2017    17423.811966
dtype: float64

In [13]:
df = pd.concat([emissions, heat_production, electricity_production, heat_share], axis=1)
df['HeatUnitEmissions'] = df.Emissions / df['Heat production'] * df['Share of heat production']
df['ElectricityUnitEmissions'] = df.Emissions / df['Electricity production'] * (1 - df['Share of heat production'])
df

,Emissions,Heat production,Electricity production,Share of heat production,HeatUnitEmissions,ElectricityUnitEmissions
Year,,,,,,
2005,3433191.804,6531.3,5266.5,0.349552,183.742875,424.022019
2010,3706427.556,7360.0,5595.5,0.363050,182.828580,421.912109
2014,3167250.696,6385.2,4824.4,0.364485,180.795343,417.220022
2015,2963375.856,5984.6,4663.5,0.357364,176.954693,408.356985
2016,3271407.372,6632.8,3976.9,0.419525,206.916714,477.500110
2017,3272854.104,6606.2,3932.6,0.421275,208.708896,481.635914


In [8]:
df = operator_fuel_with_co2.set_index('Year')
last_year = df.loc[df.index.max()]
last_year = last_year[~last_year.StatfiFuelCode.isna()]
all_fuels = last_year.Value.pint.m.sum()
last_year['Share'] = last_year.Value / all_fuels
last_year

target_year_shares = {
    'Puupelletit ja -briketit': 30,
    'Kivihiili ja antrasiitti': 30,
    
}

In [ ]:
from quilt.